In [2]:
# =============================================================================
# 步驟一：載入函式庫與設定環境
# =============================================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error, r2_score

# --- 設定 Matplotlib 顯示中文字體 ---
# 確保您的系統有支援的字體，否則圖表標題可能會顯示亂碼
try:
    # For Windows
    plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei'] 
    plt.rcParams['axes.unicode_minus'] = False # 解決負號顯示問題
except:
    try:
        # For Mac
        plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']
        plt.rcParams['axes.unicode_minus'] = False
    except:
        print("未找到指定的中文字體，圖表標題可能無法正常顯示。")

In [3]:
# =============================================================================
# 步驟二：資料載入與清理 (Data Loading & Cleaning)
# =============================================================================
# 使用您提供的新檔名 'k04.csv'
file_path = 'k04.csv' 
try:
    # 嘗試用 utf-8-sig 來讀取，它可以處理帶有 BOM 的 UTF-8 檔案
    df = pd.read_csv(file_path, encoding='utf-8-sig')
except UnicodeDecodeError:
    # 如果失敗，嘗試用 big5 編碼
    df = pd.read_csv(file_path, encoding='big5')

print("--- 資料清理開始 ---")
print(f"原始資料筆數: {len(df)}")

--- 資料清理開始 ---
原始資料筆數: 5013


In [26]:
# 1. 定義時間轉換函數
def parse_time_to_seconds(time_str):
    """將 MM:SS 或 HH:MM:SS 格式的字串轉為總秒數，錯誤格式返回 None"""
    try:
        return pd.to_timedelta(time_str).total_seconds()
    except (ValueError, TypeError):
        return None
# --------------------------------------------------------

In [27]:
# 2. 應用函數，並建立新欄位 'TotalSeconds'
df['TotalSeconds'] = df['Time2'].apply(parse_time_to_seconds)


In [28]:
print(f"下條件前清理前資料筆數: {len(df)}")


下條件前清理前資料筆數: 5013


In [31]:
# 3. 移除轉換失敗的資料列 (TotalSeconds 是 NaT 或 None)
df = df.dropna(subset=['TotalSeconds'], inplace=True)
print(f"下條件清理前資料筆數: {len(df)}")

TypeError: object of type 'NoneType' has no len()

In [30]:
# 3. 移除轉換失敗的資料列 (TotalSeconds 是 NaT 或 None)
df.dropna(subset=['TotalSeconds'], inplace=True)
print(f"下條件清理前資料筆數: {len(df)}")

df['TotalSeconds'] = df['TotalSeconds'].astype(int)
print(f"下條件前清理前資料筆數: {len(df)}")


下條件清理前資料筆數: 0
下條件前清理前資料筆數: 0


In [16]:
# 4. 移除不合理的極端值（騎乘時間小於3分鐘或大於60分鐘的資料）
df = df[(df['TotalSeconds'] >= 180) & (df['TotalSeconds'] <= 3600)].copy()
print(f"清理後剩餘資料筆數: {len(df)}")


清理後剩餘資料筆數: 0


In [ ]:
# =============================================================================
# 步驟二：資料載入與清理 (Data Loading & Cleaning)
# =============================================================================
# 使用您提供的新檔名 'k04.csv'
file_path = 'k04.csv' 
try:
    # 嘗試用 utf-8-sig 來讀取，它可以處理帶有 BOM 的 UTF-8 檔案
    df = pd.read_csv(file_path, encoding='utf-8-sig')
except UnicodeDecodeError:
    # 如果失敗，嘗試用 big5 編碼
    df = pd.read_csv(file_path, encoding='big5')

print("--- 資料清理開始 ---")
print(f"原始資料筆數: {len(df)}")
# ---------------------------------------------

# --------------------------------------------------------

# 1. 定義時間轉換函數
def parse_time_to_seconds(time_str):
    """將 MM:SS 或 HH:MM:SS 格式的字串轉為總秒數，錯誤格式返回 None"""
    try:
        return pd.to_timedelta(time_str).total_seconds()
    except (ValueError, TypeError):
        return None
# --------------------------------------------------------





# 2. 應用函數，並建立新欄位 'TotalSeconds'
df['TotalSeconds'] = df['Time2'].apply(parse_time_to_seconds)

# 3. 移除轉換失敗的資料列 (TotalSeconds 是 NaT 或 None)
df.dropna(subset=['TotalSeconds'], inplace=True)
df['TotalSeconds'] = df['TotalSeconds'].astype(int)

# 4. 移除不合理的極端值（騎乘時間小於3分鐘或大於60分鐘的資料）
df = df[(df['TotalSeconds'] >= 180) & (df['TotalSeconds'] <= 3600)].copy()
print(f"清理後剩餘資料筆數: {len(df)}")

# 5. 處理日期與時間欄位
df['Date'] = df['Date'].astype(str)
df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time1'], format='%Y%m%d %H:%M:%S')

# 6. 標準化起點站 ('Stp1')
def normalize_exit(stp_name):
    stp_name = str(stp_name)
    if '1號' in stp_name:
        return 'Exit_1'
    # 包含 '2號' 或 '_1' 的都視為2號出口
    elif '2號' in stp_name or '_1' in stp_name:
        return 'Exit_2'
    else:
        # 處理沒有標示出口的情況，暫且歸為2號
        return 'Exit_2'

df['StartExit'] = df['Stp1'].apply(normalize_exit)

print("--- 資料清理完成 ---\n")


--- 資料清理開始 ---
原始資料筆數: 5013
清理後剩餘資料筆數: 0
--- 資料清理完成 ---



In [ ]:
# =============================================================================
# 步驟三：特徵工程 (Feature Engineering)
# =============================================================================
print("--- 特徵工程開始 ---")
# 1. 從 'Datetime' 欄位中提取時間特徵
df['Hour'] = df['Datetime'].dt.hour
df['DayOfWeek'] = df['Datetime'].dt.dayofweek  # 星期一=0, 星期日=6
df['Month'] = df['Datetime'].dt.month
df['IsWeekend'] = df['DayOfWeek'].apply(lambda x: 1 if x >= 5 else 0)

# 2. 建立尖峰時段特徵
def is_rush_hour(hour):
    if (7 <= hour <= 9) or (17 <= hour <= 19):
        return 1
    return 0
df['IsRushHour'] = df['Hour'].apply(is_rush_hour)

# 3. (推薦作法) 使用 OneHotEncoder 處理類別特徵 'StartExit'
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False, dtype=int)
encoded_features = ohe.fit_transform(df[['StartExit']])
new_feature_names = ohe.get_feature_names_out(['StartExit'])
encoded_df = pd.DataFrame(encoded_features, columns=new_feature_names, index=df.index)

# 4. 將編碼後的新欄位合併回原 DataFrame，並移除舊的類別欄位
df = pd.concat([df, encoded_df], axis=1)
df.drop(['Stp1', 'StartExit'], axis=1, inplace=True) # 順便移除原始的 Stp1

print("--- 特徵工程完成 ---\n")



--- 特徵工程開始 ---


ValueError: Found array with 0 sample(s) (shape=(0,)) while a minimum of 1 is required.

In [4]:
from sklearn.preprocessing import OneHotEncoder

# --- Step 3: 特徵工程 (修正版) ---

# 從 'Datetime' 欄位中提取更多特徵
df['Hour'] = df['Datetime'].dt.hour
df['DayOfWeek'] = df['Datetime'].dt.dayofweek  # 星期一=0, 星期日=6
df['Month'] = df['Datetime'].dt.month
df['IsWeekend'] = df['DayOfWeek'].apply(lambda x: 1 if x >= 5 else 0)

# 建立尖峰時段特徵
def is_rush_hour(hour):
    if (7 <= hour <= 9) or (17 <= hour <= 19):
        return 1
    return 0
df['IsRushHour'] = df['Hour'].apply(is_rush_hour)

# --- 使用 OneHotEncoder 處理 'StartExit' ---
# 1. 實例化 OneHotEncoder
#    handle_unknown='ignore' 可以在未來預測時忽略未見過的類別
#    sparse_output=False 讓它直接輸出 numpy array 而不是稀疏矩陣
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# 2. Fit and transform the 'StartExit' column
#    需要將 df[['StartExit']] 轉為 2D array
encoded_features = ohe.fit_transform(df[['StartExit']])

# 3. 獲取新欄位的名稱 (例如 'StartExit_Exit_1', 'StartExit_Exit_2')
new_feature_names = ohe.get_feature_names_out(['StartExit'])

# 4. 將編碼後的特徵轉換為 DataFrame
encoded_df = pd.DataFrame(encoded_features, columns=new_feature_names, index=df.index)

# 5. 將新的編碼欄位合併回原始 DataFrame，並刪除舊的 'StartExit'
df = pd.concat([df, encoded_df], axis=1)
df.drop('StartExit', axis=1, inplace=True)


# --- 檢視我們建立好的特徵 ---
# 由於欄位名稱改變了，我們也更新一下要顯示的列表
print("\n最終用於建模的特徵欄位預覽：")
features_to_show = ['TotalSeconds', 'Hour', 'DayOfWeek', 'IsWeekend', 'IsRushHour'] + list(new_feature_names)
print(df[features_to_show].head())

# --- 更新後續訓練時使用的特徵列表 ---
# 在 Step 4 中，features 列表也需要更新
features = ['Hour', 'DayOfWeek', 'Month', 'IsWeekend', 'IsRushHour'] + list(new_feature_names)
print("\n用於訓練的特徵:", features)

KeyError: 'Datetime'